In [2]:
import sys
from pathlib import Path
import json
import numpy as np  
import copy
import os

parent_dir = Path("/home/ujx4ab/ondemand/CBM_Final_Project/src/LLM")
sys.path.append(str(parent_dir))
parent_dir = Path("/home/ujx4ab/ondemand/CBM_Final_Project/src/Util")
sys.path.append(str(parent_dir))

from data_utils import load_reddit_posts, get_comments_for_post, get_user_comment_history, get_comments_in_post_thread, upload_json_to_file
from Mistral.minstral_utils import generate_mistral_prompt_zero_shot
from posttree import PostTree, UserCommentHistories


/home/ujx4ab/ondemand/CBM_Final_Project/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from minstral_utils import load_quantized_model, initialize_tokenizer
model_name = 'mistralai/Mistral-7B-Instruct-v0.3'
model = load_quantized_model(model_name)
tokenizer = initialize_tokenizer(model_name)

def run_prompt_on_mistral(prompt):
    stop_token_ids = [0]
    encoded = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)

    return decoded[0].split("[/INST]", 1)[1].strip()
        

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [00:16<00:00,  5.66s/it]


In [4]:
post_tree = PostTree(
    '/home/ujx4ab/ondemand/CBM_Final_Project/Data/arcticshift/processed/r_books_posts.csv',
    '194vvqz'
    )
post_tree.create_comment_tree('/home/ujx4ab/ondemand/CBM_Final_Project/Data/arcticshift/cleaned/r_books_comments_cleaned_and_pruned.csv')
post_folder = f"/home/ujx4ab/ondemand/CBM_Final_Project/Data/eval_results/zero_shot/{post_tree.post_id}"
os.makedirs(post_folder, exist_ok=True)  # Create a folder named after the post_id
prompt_response_file = os.path.join(post_folder, "prompt_response.txt")

# tree to be updated with generated comments
new_tree = copy.deepcopy(post_tree)

user_histories = UserCommentHistories(
    '/home/ujx4ab/ondemand/CBM_Final_Project/Data/arcticshift/cleaned/r_books_comments_cleaned.csv',
    '194vvqz'
    )

list_lengths = [len(lst) for lst in user_histories.user_histories.values()]
mean_length = np.mean(list_lengths)
std_length = np.std(list_lengths)
print(f"Mean length: {mean_length}")
print(f"Standard deviation: {std_length}")


Mean length: 19.979942693409743
Standard deviation: 76.30369985807438


In [5]:
for node in post_tree.bfs_generator():
    # get the current comment and 10 of their history comments selected at random (if they have 10)
    cur_user_comment_history = user_histories.get_random_user_history(f"{node.user}")

    # get  previous responses on thread
    previous_comments = node.get_previous_responses()

    # get comment to be responded to and delete from response history
    print(previous_comments)
    try: 
        if len(previous_comments) > 0:
            comment_of_interest = previous_comments[-1]
        else:
            comment_of_interest = ""
        previous_comments = "\n".join(previous_comments)
    except TypeError: 
        comment_of_interest = ""
        previous_comments = ""
    if previous_comments != [] and node.user == 'zoapcfr':
        print(node.user, node.comment_id, cur_user_comment_history)
        print(previous_comments)
        print()
        #print(previous_comments)
        print("NEXT: \n")

    # prompt = generate_mistral_prompt_zero_shot(
    #     post_title=post_tree.title, 
    #     post_content=post_tree.content, 
    #     previous_comments_on_thread=previous_comments, 
    #     comment_of_interest=comment_of_interest,
    #     user_comment_history=cur_user_comment_history, 
    # )
    
    # response = run_prompt_on_mistral(prompt)
    # node.comment_text = response

    # with open(prompt_response_file, "a") as file:
    #     file.write(f"{prompt}\n")
    #     file.write(f"{response}\n")
    
    #print(response)    

post_tree.save_as_json(f"/home/ujx4ab/ondemand/CBM_Final_Project/Data/posttrees/{post_tree.post_id}2.json")

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[User: ColeVi123 (ID: khiqd24)
	User: According_Bat_8150 (ID: khir6vm)
		User: ColeVi123 (ID: khiro2k)
			User: VokN (ID: khixb3c)
		User: AcademyJinx (ID: khj8q03)
		User: DukeSilverPlaysHere (ID: khjhxl8)
		User: Blurghblagh (ID: khk979m)
		User: rebelbydesign (ID: khkadsd)
	User: interstatebus (ID: khj6xh4)
	User: CookieSquire (ID: khj9lym)
		User: iverybadatnames (ID: khjpkvi)
			User: Super_Nerd92 (ID: khjwt71)
				User: iverybadatnames (ID: khk1owc)
			User: Smooth-Review-2614 (ID: khqqz0h)
		User: Unlucky-Library-9030 (ID: khk9kul)
			User: CookieSquire (ID: khkdwc1)
		User: Creator13 (ID: khkoic2)
			User: Audio-et-Loquor (ID: khog9by)
				User: Creator13 (ID: khoiweu)
		User: RPBiohazard (ID: khlf53n)
	User: hotsauceandburrito (ID: khjb14k)
	User: math-is-magic (ID: khjd0hk)
		User: ColeVi123 (ID: khjeg